In [1]:
!pip install tabtransformertf --quiet

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score

%matplotlib inline

/usr/local/lib/python3.6/site-packages/google/auth/crypt/_cryptography_rsa.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  import cryptography.exceptions


In [3]:
from tabtransformertf.models.tabtransformer import TabTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset, build_categorical_prep

In [4]:
import absl.logging
import warnings
import logging

logging.captureWarnings(True)
warnings.filterwarnings('ignore')
absl.logging.set_verbosity(absl.logging.ERROR)

In [5]:
train_data = pd.read_csv('fraud_train_preprocessed.csv')
val_data = pd.read_csv('fraud_val_preprocessed.csv')

## Preprocessing

In [6]:
# Column information
NUMERIC_FEATURES = train_data.select_dtypes(include=np.number).columns[:-2] # exclude label column and DT
CATEGORICAL_FEATURES = train_data.select_dtypes(exclude=np.number).columns

FEATURES = list(NUMERIC_FEATURES) + list(CATEGORICAL_FEATURES)
LABEL = 'isFraud'

In [7]:
# Set data types
train_data[CATEGORICAL_FEATURES] = train_data[CATEGORICAL_FEATURES].astype(str)
val_data[CATEGORICAL_FEATURES] = val_data[CATEGORICAL_FEATURES].astype(str)

train_data[NUMERIC_FEATURES] = train_data[NUMERIC_FEATURES].astype(float)
val_data[NUMERIC_FEATURES] = val_data[NUMERIC_FEATURES].astype(float)

In [8]:
category_prep_layers = build_categorical_prep(train_data, CATEGORICAL_FEATURES)

100%|██████████| 46/46 [00:02<00:00, 21.54it/s]


In [9]:
import gc
gc.collect()

0

### TF Dataset

In [10]:
train_dataset = df_to_dataset(train_data[FEATURES + [LABEL]], LABEL, batch_size=3072)
val_dataset = df_to_dataset(val_data[FEATURES + [LABEL]], LABEL, shuffle=False, batch_size=3072)

## TabTransformer

In [23]:
tabtransformer = TabTransformer(
    numerical_features = NUMERIC_FEATURES,
    categorical_features = CATEGORICAL_FEATURES,
    categorical_lookup=category_prep_layers,
    numerical_discretisers=None, # simply passing the numeric features
    embedding_dim=32,
    out_dim=1,
    out_activation='sigmoid',
    depth=4,
    heads=8,
    attn_dropout=0.1,
    ff_dropout=0.1,
    mlp_hidden_factors=[2, 4],
    use_column_embedding=True,
)

In [26]:
tabtransformer.summary()

Model: "tab_transformer_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_9 (Layer multiple                  270       
_________________________________________________________________
concatenate_3 (Concatenate)  multiple                  0         
_________________________________________________________________
string_lookup (StringLookup) multiple                  0         
_________________________________________________________________
string_lookup_1 (StringLooku multiple                  0         
_________________________________________________________________
string_lookup_2 (StringLooku multiple                  0         
_________________________________________________________________
string_lookup_3 (StringLooku multiple                  0         
_________________________________________________________________
string_lookup_4 (StringLooku multiple            

In [24]:
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.0001
NUM_EPOCHS = 1000

optimizer = tfa.optimizers.AdamW(
        learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

tabtransformer.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics= [tf.keras.metrics.AUC(name="PR AUC", curve='PR')],
)

In [25]:
out_file = './tabTransformerBasic'
checkpoint = ModelCheckpoint(
    out_file, monitor="val_loss", verbose=1, save_best_only=True, mode="min"
)
early = EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)
callback_list = [checkpoint, early]

history = tabtransformer.fit(
    train_dataset, 
    epochs=NUM_EPOCHS, 
    validation_data=val_dataset,
    callbacks=callback_list
)

Epoch 1/1000
174/174 [==============================] - 128s 404ms/step - loss: 0.1581 - PR AUC: 0.2005 - val_loss: 0.1525 - val_PR AUC: 0.3457

Epoch 00001: val_loss improved from inf to 0.15253, saving model to ./tabTransformerBasic
INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


Epoch 2/1000
174/174 [==============================] - 113s 392ms/step - loss: 0.1132 - PR AUC: 0.3807 - val_loss: 0.1293 - val_PR AUC: 0.3911

Epoch 00002: val_loss improved from 0.15253 to 0.12931, saving model to ./tabTransformerBasic
INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


Epoch 3/1000
174/174 [==============================] - 112s 408ms/step - loss: 0.1086 - PR AUC: 0.4129 - val_loss: 0.1245 - val_PR AUC: 0.4164

Epoch 00003: val_loss improved from 0.12931 to 0.12446, saving model to ./tabTransformerBasic
INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


...

Epoch 24/1000
174/174 [==============================] - 110s 404ms/step - loss: 0.0866 - PR AUC: 0.5847 - val_loss: 0.1114 - val_PR AUC: 0.4472

Epoch 00024: val_loss did not improve from 0.10835
Epoch 25/1000
174/174 [==============================] - 110s 403ms/step - loss: 0.0861 - PR AUC: 0.5892 - val_loss: 0.1086 - val_PR AUC: 0.4518

Epoch 00025: val_loss did not improve from 0.10835
Epoch 26/1000
174/174 [==============================] - 109s 401ms/step - loss: 0.0865 - PR AUC: 0.5857 - val_loss: 0.1073 - val_PR AUC: 0.4620

Epoch 00026: val_loss improved from 0.10835 to 0.10732, saving model to ./tabTransformerBasic
INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


Epoch 27/1000
174/174 [==============================] - 107s 397ms/step - loss: 0.0856 - PR AUC: 0.5917 - val_loss: 0.1085 - val_PR AUC: 0.4583

Epoch 00027: val_loss did not improve from 0.10732
Epoch 28/1000
174/174 [==============================] - 109s 405ms/step - loss: 0.0850 - PR AUC: 0.5975 - val_loss: 0.1134 - val_PR AUC: 0.4329

Epoch 00028: val_loss did not improve from 0.10732
Epoch 29/1000
174/174 [==============================] - 108s 402ms/step - loss: 0.0852 - PR AUC: 0.5958 - val_loss: 0.1145 - val_PR AUC: 0.4264

Epoch 00029: val_loss did not improve from 0.10732
Epoch 30/1000
174/174 [==============================] - 110s 408ms/step - loss: 0.0849 - PR AUC: 0.5980 - val_loss: 0.1062 - val_PR AUC: 0.4701

Epoch 00030: val_loss improved from 0.10732 to 0.10615, saving model to ./tabTransformerBasic
INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


INFO:tensorflow:Assets written to: ./tabTransformerBasic/assets


Epoch 31/1000
174/174 [==============================] - 109s 399ms/step - loss: 0.0837 - PR AUC: 0.6070 - val_loss: 0.1152 - val_PR AUC: 0.4020

Epoch 00031: val_loss did not improve from 0.10615
Epoch 32/1000
174/174 [==============================] - 109s 399ms/step - loss: 0.0845 - PR AUC: 0.6012 - val_loss: 0.1093 - val_PR AUC: 0.4524

Epoch 00032: val_loss did not improve from 0.10615
Epoch 33/1000
174/174 [==============================] - 108s 400ms/step - loss: 0.0827 - PR AUC: 0.6139 - val_loss: 0.1118 - val_PR AUC: 0.4541

Epoch 00033: val_loss did not improve from 0.10615
Epoch 34/1000
174/174 [==============================] - 109s 400ms/step - loss: 0.0844 - PR AUC: 0.6005 - val_loss: 0.1087 - val_PR AUC: 0.4615

Epoch 00034: val_loss did not improve from 0.10615
Epoch 35/1000
174/174 [==============================] - 111s 405ms/step - loss: 0.0843 - PR AUC: 0.6009 - val_loss: 0.1098 - val_PR AUC: 0.4607

Epoch 00035: val_loss did not improve from 0.10615
Epoch 36/1000
1

In [13]:
tabtransformer = tf.keras.models.load_model('./tabTransformerBasic')

In [12]:
val_preds = tabtransformer.predict(val_dataset)

print(f"PR AUC: {average_precision_score(val_data['isFraud'], val_preds.ravel())}")
print(f"ROC AUC: {roc_auc_score(val_data['isFraud'], val_preds.ravel())}")

PR AUC: 0.4701335294324417
ROC AUC: 0.8794082315520849


## Test set prediction

In [9]:
test_data = pd.read_csv('fraud_test_preprocessed.csv')

In [10]:
test_data[CATEGORICAL_FEATURES] = test_data[CATEGORICAL_FEATURES].astype(str)
test_data[NUMERIC_FEATURES] = test_data[NUMERIC_FEATURES].astype(float)

In [11]:
test_dataset = df_to_dataset(test_data[FEATURES], None, shuffle=False, batch_size=56)

In [14]:
test_preds = tabtransformer.predict(test_dataset)

In [15]:
transaction_ids = pd.read_csv('../test_transaction.csv', usecols=['TransactionID'])
submission_df = pd.DataFrame({"TransactionID": transaction_ids.values.ravel(),
              "isFraud": test_preds.ravel()
             })
submission_df.to_csv("tabtransformer_basic_submission.csv", index=False)

In [23]:
submission_df.sample(4)

,TransactionID,isFraud
145394,3808943,0.025497
378395,4041944,0.007631
48681,3712230,0.028034
279758,3943307,0.025208
